<a href="https://www.kaggle.com/code/esracum/thermal-image-process-with-cpp?scriptVersionId=290960720" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os

# Termal görüntülerin olduğu klasörü bulalım
# (Klasör ismi veri setine göre değişebilir, bu kod onu bulur)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith(".tiff") or filename.endswith(".raw"):
            print(os.path.join(dirname, filename))
            break # Sadece ilk dosyayı göster yeter

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <opencv2/opencv.hpp> // Görüntü işleme kütüphanesi

using namespace std;
using namespace cv;

int main() {
    // 1. Dosya Yolu 
    string dosyaYoluI = "/kaggle/input/teledyne-flir-adas-thermal-dataset-v2/FLIR_ADAS_v2/images_thermal_train/analyticsData/video-24ysbPEGoEKKDvRt6-frame-000000-4C4FHWxwNaMyohLZt.tiff"; 
    string dosyaYoluR =          "/kaggle/input/teledyne-flir-adas-thermal-dataset-v2/FLIR_ADAS_v2/images_thermal_train/data/video-24ysbPEGoEKKDvRt6-frame-000000-4C4FHWxwNaMyohLZt.jpg";

    // 2. Bu dosyayı açarken sakın sıkıştırma, yuvarlama veya kırpma yapma. Dosyayı olduğu gibi oku.
    // IMREAD_ANYDEPTH: Çok önemli! Bunu demezsen 8-bit okur, veriyi bozar.
    Mat termalGoruntu = imread(dosyaYolu, IMREAD_ANYDEPTH);

    // 3. Kontrol Et
    if (termalGoruntu.empty()) {
        cout << "HATA: Dosya bulunamadi veya acilamadi!" << endl;
        return -1;
    }

    // 4. Veri Analizi 
    cout << "--- TERMAL VERI ANALIZI ---" << endl;
    cout << "Genislik: " << termalGoruntu.cols << " piksel" << endl;
    cout << "Yukseklik: " << termalGoruntu.rows << " piksel" << endl;
    
    // Veri tipini kontrol et (CV_16U olmalı, yani 16-bit Unsigned)
    cout << "Bit Derinligi: " << termalGoruntu.depth() << " (2 ise 16-bit demektir)" << endl;

    // 5. En Sıcak ve En Soğuk Noktayı Bul
    double minVal, maxVal;
    minMaxLoc(termalGoruntu, &minVal, &maxVal);

    cout << "En Soguk Deger (Sensor Verisi): " << minVal << endl;
    cout << "En Sicak Deger (Sensor Verisi): " << maxVal << endl;

    if (maxVal > 255) {
        cout << "SONUC: Basarili! 8-bit degil, gercek 16-bit ham veri okuyorsun." << endl;
    } else {
        cout << "UYARI: Bu veri 8-bit gibi gorunuyor. Yanlis dosya olabilir." << endl;
    }

    return 0;
}

In [ ]:
# 1. Önce kodu derleyelim (OpenCV kütüphanesini bağlayarak)
!g++ main.cpp -o main `pkg-config --cflags --libs opencv4`

# 2. Eğer derleme hatasız biterse, programı çalıştıralım
!./main

In [ ]:
%%writefile main.cpp
#include <iostream>
#include <opencv2/opencv.hpp>

using namespace std;
using namespace cv;

int main() {
    // 1. DOSYALARI TANIMLA
    // Senin verdiğin yollar:
    string yolTermal = "/kaggle/input/teledyne-flir-adas-thermal-dataset-v2/FLIR_ADAS_v2/images_thermal_train/analyticsData/video-24ysbPEGoEKKDvRt6-frame-000000-4C4FHWxwNaMyohLZt.tiff";
    string yolRGB    = "/kaggle/input/teledyne-flir-adas-thermal-dataset-v2/FLIR_ADAS_v2/images_thermal_train/data/video-24ysbPEGoEKKDvRt6-frame-000000-4C4FHWxwNaMyohLZt.jpg"; 
    // NOT: İleride yolRGB yi gerçek RGB klasöründen seçmelisin. Şimdilik deneme yapıyoruz.

    // 2. GÖRÜNTÜLERİ OKU 
    
    Mat imgTermalRaw = imread(yolTermal, IMREAD_ANYDEPTH); // 16-bit oku
    Mat imgRGB = imread(yolRGB); // Renkli oku

    if (imgTermalRaw.empty() || imgRGB.empty()) {
        cout << "Hata: Dosyalar okunamadi!" << endl;
        return -1;
    }

    // --- 3. TERMAL VERİYİ GÖRÜNÜR YAP (NORMALİZASYON) ---
    // 16-bit veriyi (0-65535), 8-bit veriye (0-255) sıkıştırıyoruz.
    Mat imgTermal8bit;
    double minVal, maxVal;
    minMaxLoc(imgTermalRaw, &minVal, &maxVal); // En sıcak ve soğuk noktayı bul
    
    // Matematiksel dönüşüm: (Deger - Min) / (Max - Min) * 255
    imgTermalRaw.convertTo(imgTermal8bit, CV_8U, 255.0/(maxVal - minVal), -minVal * 255.0/(maxVal - minVal));

    // Termal görüntü siyah-beyaz olmasın, "Isı Haritası" renkleri verelim (Opsiyonel ama havalı)
    Mat imgTermalRenkli;
    applyColorMap(imgTermal8bit, imgTermalRenkli, COLORMAP_JET);

    // --- 4. BOYUTLARI EŞİTLE (RESIZE) ---
    // RGB resmin boyutu neyse, Termal resmi de o boyuta getir (Büyüt/Küçült)
    resize(imgTermalRenkli, imgTermalRenkli, imgRGB.size());

    // --- 5. FÜZYON (KARIŞTIRMA) ---
    Mat imgFusion;
    double alpha = 0.6; // %60 RGB
    double beta = 0.4;  // %40 Termal
    
    // Formül: Sonuc = (RGB * alpha) + (Termal * beta) + 0
    addWeighted(imgRGB, alpha, imgTermalRenkli, beta, 0.0, imgFusion);

    // --- 6. KAYDET ---
    imwrite("Fusion_Sonuc.jpg", imgFusion);
    
    cout << "Basarili! 'Fusion_Sonuc.jpg' olusturuldu." << endl;
    cout << "RGB Boyut: " << imgRGB.cols << "x" << imgRGB.rows << endl;
    cout << "Termal Boyut: " << imgTermalRaw.cols << "x" << imgTermalRaw.rows << " -> " << imgRGB.cols << "x" << imgRGB.rows << " (Buyutuldu)" << endl;

    return 0;
}

In [ ]:
!g++ main.cpp -o main `pkg-config --cflags --libs opencv4`
!./main

In [ ]:
import matplotlib.pyplot as plt
import cv2

img = cv2.imread("Fusion_Sonuc.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Renkleri düzelt

plt.figure(figsize=(10,10))
plt.imshow(img)
plt.title("Sensor Fusion Sonucu (C++)")
plt.axis('off')
plt.show()